# qecsim demos

## Simulating error correction with a rotated planar stabilizer code
This demo shows verbosely how to simulate one error correction run. 

| For normal use, the code in this demo is encapsulated in the function:
| `qecsim.app.run_once(code, error_model, decoder, error_probability)`,
| and the simulation of many error correction runs is encapsulated in the function:
| `qecsim.app.run(code, error_model, decoder, error_probability, max_runs, max_failures)`.

Notes:

* Operators can be visualised in binary symplectic form (bsf) or Pauli form, e.g. `[1 1 0|0 1 0] = XYI`.
* The binary symplectic product is denoted by $\odot$ and defined as $A \odot B \equiv A \Lambda B \bmod 2$ where $\Lambda = \left[\begin{matrix} 0 & I \\ I & 0 \end{matrix}\right]$.
* Binary addition is denoted by $\oplus$ and defined as addition modulo 2, or equivalently exclusive-or.

### Initialise the models

In [1]:
%run qsu.ipynb  # color-printing functions
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

# initialise models
my_code = RotatedPlanarCode(7, 7)
my_error_model = DepolarizingErrorModel()
my_decoder = RotatedPlanarMPSDecoder(chi=8)
# print models
print(my_code)
print(my_error_model)
print(my_decoder)

Setting permissions of DOT_SAGE directory so only you can read and write it.
RotatedPlanarCode(7, 7)
DepolarizingErrorModel()
RotatedPlanarMPSDecoder(8, 'c', None)


In [51]:
code = RotatedPlanarCode(6,6)
print(code.site_bounds)
code_index=(0,0)
code.is_z_plaquette(code_index)
def _rotate_q_index(index, code):
    """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
    site_x, site_y = index  # qubit index in (x, y)
    site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
    print(site_r, site_c)
    return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

_rotate_q_index(code_index, code)

(5, 5)
5 0


(10, 5)

In [43]:
tn_max_r, _ = _rotate_q_index((0, 0), code)
_, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)
tn = np.empty((tn_max_r + 1, tn_max_c + 1), dtype=object)
tn.shape

5 0
5 5


(11, 11)

### Generate a random error

In [70]:
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

In [77]:
# set physical error probability to 10%
error_probability = 0.4
bias =0.5
# seed random number generator for repeatability
rng = np.random.default_rng(10)
error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
decoder = RotatedPlanarMPSDecoder_def(chi=chi_val)
    
# error: random error based on error probability
error = error_model.generate(code, error_probability, rng)
qsu.print_pauli('error:\n{}'.format(code.new_pauli(error)))

### Evaluate the syndrome
The syndrome is a binary array indicating the stabilizers with which the error does not commute. It is calculated as $syndrome = error \odot stabilisers^T$.

In [31]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

### Find a recovery operation
In this case, the recovery operation is found by contracting a tensor network defined to have the value of the coset :

* Create a sample recovery operation by applying strings of Paulis between syndrome plaquettes and appropriate boundaries.
* Define tensor networks corresponding to the probability of each left coset of the generator group with the sample recovery and logical Pauli operations.
* Contract each tensor network (approximately) to evaluate the coset probabilities.
* Return any recovery from the most probable coset.

In [32]:
# recovery: best match recovery operation based on decoder
recovery = my_decoder.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

As a sanity check, we expect $recovery \oplus error$ to commute with all stabilizers, i.e. $(recovery \oplus error) \odot stabilisers^T = 0$.

In [33]:
# check recovery ^ error commutes with stabilizers (by construction)
print(pt.bsp(recovery ^ error, my_code.stabilizers.T))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Visualise $recovery \oplus error$
Just out of curiosity, we can see what $recovery \oplus error$ looks like. If successful, it should be a product of stabilizer plaquette operators.

In [34]:
# print recovery ^ error (out of curiosity)
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

### Test if the recovery operation is successful
The recovery operation is successful iff $recovery \oplus error$ commutes with all logical operators, i.e. $(recovery \oplus error) \odot logicals^T = 0.$

In [35]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(recovery ^ error, my_code.logicals.T))

[0 0]


Note: The decoder is not guaranteed to find a successful recovery operation. The rotated planar 7 x 7 code has distance $d = 7$ so we can only guarantee to correct errors up to weight $(d - 1)/2=3$.

### Equivalent code in single call
The above demo is equivalent to the following code.

In [4]:
# repeat demo in single call
from qecsim import app
print(app.run_once(my_code, my_error_model, my_decoder, error_probability))

{'error_weight': 7, 'success': True, 'logical_commutations': array([0, 0]), 'custom_values': None}


In [17]:
%matplotlib inline
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from _planarmpsdecoder_def import PlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
sizes= range(5,9,2)
codes_and_size = [PlanarCode(*(size,size)) for size in sizes]

In [18]:
codes_and_size

[PlanarCode(5, 5), PlanarCode(7, 7)]

In [19]:
x=np.zeros(5)

In [20]:
x=np.append(x,1)

In [21]:
x

array([0., 0., 0., 0., 0., 1.])

In [23]:
for code_index in itertools.product(range(-1, 5 + 1), range(-1, 5 + 1)):
    print(code_index)

(-1, -1)
(-1, 0)
(-1, 1)
(-1, 2)
(-1, 3)
(-1, 4)
(-1, 5)
(0, -1)
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(1, -1)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(3, -1)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(4, -1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(5, -1)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)


In [ ]:
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

def parallel_step_p(code,hadamard_mat,error_model, decoder, max_runs,error_probability):
    result= app_def.run_def(code,hadamard_mat, error_model, decoder, error_probability, max_runs)
    return result

# set models
sizes= range(5,10,4)
codes_and_size = [RotatedPlanarCode(*(size,size)) for size in sizes]
bias_list=[30,100000]

code_name="optimal"
code_name="random"
code_name="CSS"
code_name="XZZX"

if (code_name=="random"):
    realizations=50
else:
    realizations=1

# set physical error probabilities
error_probability_min, error_probability_max = 0.05, 0.24
error_probabilities = np.linspace(error_probability_min, error_probability_max, 10)
# set max_runs for each probability
max_runs = 2000

timestr = time.strftime("%Y%m%d-%H%M%S ")   #record current date and time
dirname="./data/"+timestr+code_name
os.mkdir(dirname)     

for bias in bias_list:
    error_model = BiasedDepolarizingErrorModel(bias,'Z')
    chi_val=10
    decoder = RotatedPlanarMPSDecoder_def(chi=chi_val)

    # print run parameters
    print('codes_and_size:', [code.label for code in codes_and_size])
    print('Error model:', error_model.label)
    print('Decoder:', decoder.label)
    print('Error probabilities:', error_probabilities)
    print('Maximum runs:', max_runs)

    pL_list_rand =np.zeros((len(codes_and_size),realizations,len(error_probabilities)))
    std_list_rand=np.zeros((len(codes_and_size),realizations,len(error_probabilities)))

    pL_list =np.zeros((len(codes_and_size),len(error_probabilities)))
    std_list=np.zeros((len(codes_and_size),len(error_probabilities)))
    def _rotate_q_index(index, code):
        """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
        site_x, site_y = index  # qubit index in (x, y)
        site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
        return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

    for code_index,code in enumerate(codes_and_size):
        tn_max_r, _ = _rotate_q_index((0, 0), code)
        _, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)

        hadamard_mat=np.zeros((tn_max_r,tn_max_c))

        if code_name=="random":
            pH=0.5  
            for realization_index in range(realizations):
                for row, col in np.ndindex(hadamard_mat.shape):
                    if ((row%2==0 and col%2==0) or (row%2==1 and col%2==1)):
                        if(np.random.rand(1,1))<pH:
                            hadamard_mat[row,col]=1

                p=mp.Pool()
                func=partial(parallel_step_p,code,hadamard_mat, error_model, decoder, max_runs)
                result=p.map(func, error_probabilities)
                #print(result)
                p.close()
                p.join()
                for i in range(len(result)):
                    pL_list_rand[code_index][realization_index][i]=result[i][0]
                    std_list_rand[code_index][realization_index][i]=result[i][1]
                
            pL_list[code_index] = np.sum(pL_list_rand[code_index],axis=0)/realizations
            std_list[code_index] = np.sum(std_list_rand[code_index],axis=0)/realizations**2

        else:
            if code_name=="XZZX":
                for row, col in np.ndindex(hadamard_mat.shape):
                    if row%2==0 and col%2==0:
                        hadamard_mat[row,col]=1

            if code_name=="optimal":
                d=hadamard_mat.shape[0]
                for row, col in np.ndindex(hadamard_mat.shape):
                    if (row ==0 and col in range(0,d,4)) or (row==d-1 and col in range(2,d-1,4)) or (row%2==1 and col%2==1):
                        hadamard_mat[row,col]=1

            p=mp.Pool()
            func=partial(parallel_step_p,code,hadamard_mat,error_model, decoder, max_runs)
            result=p.map(func, error_probabilities)
            print(result)
            p.close()
            p.join()
            
            for i in range(len(result)):
                pL_list[code_index][i]=result[i][0]   
                std_list[code_index][i]=result[i][1]

        
    plt.figure(figsize=(20,10))
    lines = ["-",":","--","-."]
    linecycler = cycle(lines)
    plt.title('TND at bias='+str(bias)+' and xi='+str(chi_val))
    for sizes_index,size in enumerate(sizes):
        plt.errorbar(error_probabilities,pL_list[sizes_index],std_list[sizes_index])
    plt.xlabel('p')
    plt.ylabel('$p_L$')
    plt.legend(sizes) 
    plt.savefig(dirname+"/threshold_plot_bias_"+str(bias)+".pdf")


In [126]:
rng = np.random.default_rng() if rng is None else rng
n_qubits =3
probability=0.8
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

p

(0.19999999999999996,
 0.26666666666666666,
 0.26666666666666666,
 0.26666666666666666)

In [132]:
error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error_pauli

'YIY'

In [133]:
pt.pauli_to_bsf(error_pauli)

array([1, 0, 1, 1, 0, 1])